# Melbourne Housing Price Prediction Analysis

### Data Exploration and Visualization

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import make_scorer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from statsmodels.regression.linear_model import OLS

In [2]:
data = pd.read_csv('Melbourne_housing.csv')
data = data.loc[:, ~data.columns.str.contains('^Unnamed')]
data

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Thornbury,7/67 Pender St,2,u,438000.0,SP,Love,22/08/2016,6.5,3071,...,1.0,1.0,0.0,66.0,1970.0,Darebin City Council,-37.75410,145.00880,Northern Metropolitan,8870
1,Coburg,32 Rose St,2,h,901000.0,S,Peter,27/06/2016,7.8,3058,...,1.0,1.0,545.0,107.0,1940.0,Darebin City Council,-37.74450,144.94710,Northern Metropolitan,11204
2,Bentleigh East,2/25 Brooks St,3,u,800000.0,S,Woodards,28/05/2016,13.9,3165,...,2.0,2.0,261.0,NaN,NaN,Glen Eira City Council,-37.91630,145.07790,Southern Metropolitan,10969
3,Templestowe Lower,2/35 John St,3,t,1155000.0,S,Jellis,25/11/2017,12.4,3107,...,NaN,NaN,NaN,NaN,NaN,Manningham City Council,NaN,NaN,Eastern Metropolitan,5420
4,South Yarra,1/35 Marne St,3,u,1630000.0,PI,Jellis,10/9/2016,3.3,3141,...,1.0,1.0,0.0,NaN,NaN,Melbourne City Council,-37.83590,144.98390,Southern Metropolitan,14887
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,Sunshine,74 Hertford Rd,2,h,965000.0,S,Barry,26/08/2017,10.5,3020,...,1.0,1.0,NaN,NaN,NaN,Brimbank City Council,-37.78227,144.84038,Western Metropolitan,3755
495,Springvale,15 Phillip Av,3,h,721000.0,S,Hall,24/06/2017,20.8,3171,...,NaN,NaN,NaN,NaN,NaN,Greater Dandenong City Council,NaN,NaN,South-Eastern Metropolitan,7412
496,Middle Park,150 Page St,3,h,1750000.0,VB,Greg,27/05/2017,3.0,3206,...,2.0,0.0,138.0,155.0,1885.0,Port Phillip City Council,-37.84962,144.95856,Southern Metropolitan,2019
497,Glenroy,14 William St,3,h,501000.0,SP,Raine,28/05/2016,13.0,3046,...,1.0,1.0,348.0,NaN,NaN,Moreland City Council,-37.71190,144.91110,Northern Metropolitan,8870


In [ ]:
data.columns

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
# find the value counts for suburbs and postcode
print('Value counts for Suburb')
print(data['Suburb'].value_counts(),'\n\n\n')

print('Value counts for Postcode')
print(data['Postcode'].value_counts())

In [ ]:
# histgram to find house pricing distribution

sns.histplot(data['Price'], bins=20, kde=True)

# Format the x-axis and/or y-axis to regular format
ax = plt.gca()  # Get the current axis
ax.xaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))

# Histgram
plt.xlabel('House Price')
plt.ylabel('Frequency')
plt.title('Histogram of Melbourne House Price')
plt.show()

In [ ]:
# find the scatter plot for house price and number of rooms

sns.scatterplot(data=data, x='Rooms', y='Price')

# change axis number format
ax = plt.gca()  
ax.yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))

plt.title('House Price vs Room Numbers')
plt.xlabel('Number of Rooms')
plt.ylabel('House Prices')

# Show the plot
plt.show()

In [ ]:
# find distribution of house prices from different property types

sns.boxplot(data=data, x='Type', y='Price')

# change axis number format
ax = plt.gca()  
ax.yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))

# Optional customization
plt.title('House Price Distribution of Different Property Types')
plt.xlabel('Property Types')
plt.ylabel('House Prices')

# Show the plot
plt.show()

Based on the visual analysis of the Melbourne Housing dataset, it is observed that most houses are listed at approximately 1 million dollars, showing a positively skewed distribution. Furthermore, there is a noticeable trend where house prices generally increase with an increase in the number of rooms, particularly from 1 to 4 rooms. However, a marginal decrease in price is observed for houses with 5 rooms. It seems that the ideal number of rooms for a property is 4 in order for property owners to yield highest values for their property assets.

Additionally, property types such as houses, cottages, villas, semis, and terraces tend to command the highest values. This is followed by townhouses, while units or duplexes are typically priced lower, making them the least expensive among the three property types.

### Linear Regression Model Development

In [5]:
# drop unnecessay columns that do not affect the house prices prediction
data.columns
columns = ['Rooms', 'Type', 'Price', 'Distance', 'Bathroom', 'Car',
       'Landsize', 'YearBuilt', 'Regionname', 'Propertycount']

data = data.loc[:,columns]

In [6]:
# fill NaN with some values rather than drop NaN values
data['Bathroom'] = data['Bathroom'].fillna(data['Bathroom'].mean())
data['Car'] = data['Car'].fillna(0.0)
data['Landsize'] = data['Landsize'].fillna(data['Landsize'].mean())
data['YearBuilt'] = data['YearBuilt'].fillna(data['YearBuilt'].mean())

In [7]:
# drop rows/columns with null values
data = data.dropna()
data = data.dropna(axis=1)
data.info()
data.reset_index(inplace = True)
data = data.drop('index',axis = 1)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 391 entries, 0 to 497
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Rooms          391 non-null    int64  
 1   Type           391 non-null    object 
 2   Price          391 non-null    float64
 3   Distance       391 non-null    float64
 4   Bathroom       391 non-null    float64
 5   Car            391 non-null    float64
 6   Landsize       391 non-null    float64
 7   YearBuilt      391 non-null    float64
 8   Regionname     391 non-null    object 
 9   Propertycount  391 non-null    int64  
dtypes: float64(6), int64(2), object(2)
memory usage: 33.6+ KB


In [8]:
# one-hot encoding for category variables
data = pd.get_dummies(data, columns=['Type','Regionname'])

In [9]:
# standardize our continuous features to unify their scales
continuous_features = ['Rooms', 'Distance', 'Bathroom', 'Car',
       'Landsize', 'YearBuilt', 'Propertycount']

scaler = StandardScaler()
data[continuous_features] = scaler.fit_transform(data[continuous_features])

In [10]:
# set target variables and features for linear regression model
price = data['Price']
feature = data.loc[:,data.columns]
feature = feature.drop('Price',axis = 1)

# split data into training and test set
x_train, x_test, y_train, y_test = train_test_split(feature, price, test_size=0.3, random_state=4)

# develop the linear regression model
model = sm.OLS(y_train,sm.add_constant(x_train)).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.622
Model:                            OLS   Adj. R-squared:                  0.598
Method:                 Least Squares   F-statistic:                     26.32
Date:                Tue, 30 Jan 2024   Prob (F-statistic):           4.43e-45
Time:                        16:36:33   Log-Likelihood:                -3902.9
No. Observations:                 273   AIC:                             7840.
Df Residuals:                     256   BIC:                             7901.
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                                            coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------

#### Steps:
- First, I dropped some of the columns that do not really help with the price predictions.
- Among the remaining columns, I filled the NA values either with 0 or the mean values of that specific column.
- Then I drop rows and columns that still contain NA values.
- After having my data ready, I started one-hot encoding for category variables for future linear regression model.
- The next step: I used StandardScaler() to standardize my continuous features to enable my model to be more accurate.
- Then I set target variables and features for linear regression model, and created training and test set for my data.
- The last step is to build the linear regression model using my training x and training data and extract the model summay.

### Model Evaluation


In [11]:
# find the predicted price values based on the model built from training data
x_test_constant = sm.add_constant(x_test)
y_pred = model.predict(x_test_constant)

In [12]:
# Evaluate model performance using the deviance and R^2 functioon created by professor
def deviance(y, pred, family="gaussian"):
    if family == "gaussian":
        return np.sum((y - pred) ** 2)
    elif family == "binomial":
        y = np.array(y)
        pred = np.array(pred)
        return -2 * np.sum(y * np.log(pred) + (1 - y) * np.log(1 - pred))

def R2(y, pred, family="gaussian"):
    dev_val = deviance(y, pred, family=family)
    dev0 = deviance(y, [np.mean(y)] * len(y), family=family)
    return 1 - dev_val / dev0

In [13]:
R_square = R2(y_test,y_pred)
R_square

0.5347567940020275

In [14]:
# OOS: Now we will use 10-fold cross validation to collect 10 R_square score, and find the mean R_square score for my model through OOS
model = LinearRegression()

fold_10 = KFold(n_splits=10, shuffle = True, random_state= 1)
r_squares = cross_val_score(model, feature, price, cv = fold_10, scoring = make_scorer(R2))

# Collect the R_squared scores and find the mean R_squared score
print("R-squares:\n", r_squares, "\n")
print("Average R-squares\n:", np.mean(r_squares))

R-squares:
 [0.44125198 0.22925798 0.44467192 0.58590602 0.67000963 0.59982905
 0.61692387 0.50339666 0.68504686 0.53107074] 

Average R-squares
: 0.5307364714117421


In [15]:
# Now we will evaluate the mean squared error
mse = cross_val_score(model, feature, price, cv = fold_10, scoring = make_scorer(mean_squared_error))

# Collect the R_squared scores and find the mean R_squared score
print("Mean Squared Error:\n", mse, "\n")
print("Average Mean Squared Error:\n", np.mean(mse))

Mean Squared Error:
 [1.83375533e+11 1.31864098e+11 5.27563608e+11 9.32490149e+10
 7.39953030e+10 1.34071324e+11 1.13556428e+11 1.84497602e+11
 2.40640883e+11 1.12829767e+11] 

Average Mean Squared Error:
 179564356172.5467


By using 10 fold cross validation to find the mean R^2 score of my linear regression model, I got a score of 0.55, which means that the model i trained can explain approximatly 55% of the prediction deviance. And the Mean Squared error indicates that the square of our prediction error is approximately 179564356172.5467.

### Regularization** (25 pts):
- Is there a need for Lasso regularization? Explain. (Compare test and training results)

- Set up a Lasso regression model with specified parameters and fit it to the training data.

- Perform Lasso regression on both training and test data

- Analyze and explain how the results change from previous answer.

In [16]:
from sklearn.linear_model import Lasso

# we first try to build a lasso model with alpha being set to 2
lasso_model = Lasso(alpha=2.0)
lasso_model.fit(x_train, y_train)

/Users/eddyyu/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.571e+12, tolerance: 1.106e+10
  model = cd_fast.enet_coordinate_descent(


Lasso(alpha=2.0)

In [17]:
# find the predicted y values from the lasso model
y_pred_lasso = lasso_model.predict(x_test)
print('First predicted result from lasso model:\n')
print(y_pred_lasso[:50],'\n\n\n')      # first 50 results

# find the coefficients and intercepts for the lasso model
print("Lasso coefficients:\n\n", lasso_model.coef_, '\n\n\n')
print("Intercept:\n\n", lasso_model.intercept_)

First predicted result from lasso model:

[1425540.95075895 1005019.61445224  992237.65551436 1622379.53315155
  921600.11088716 1424464.2746963   866613.08224081  747301.82531493
  910653.34362872 1220603.55496609  760034.90502415  271572.54017417
 1050108.2258804  1739868.87703999 -103045.19712047  840128.27131303
    5905.50198925  501755.13298072  758784.70459464 1023890.26237956
 1527568.38587719 1019128.07659852 1191591.71625038 1232937.05373457
  170135.83371659 1367491.27564809 2468030.48045165 1266084.6541577
 1157382.29682054 1102042.67503105 2316088.66848866  227061.73437831
 1829697.88246575 1290418.5221836   447369.62303696 1718242.25540377
  794134.72977224 1130609.35356543  758282.37203854  285579.06885504
  492691.14030551  773719.14605732  421333.4650997  1289896.11079527
  721764.18562557  977735.7025762   647501.56131562 1002324.90237367
  880059.56185509  527686.34433813] 



Lasso coefficients:

 [ 163447.36704974 -270339.84695831  141636.82569103   83479.09672543


In [18]:
# find within sample R^2
R_square_lasso = R2(y_test,y_pred_lasso)
R_square_lasso

# OOS: Now we will use 10-fold cross validation to collect 10 R_square score, and find the mean R_square score for my model through OOS
fold_10 = KFold(n_splits=10, shuffle = True, random_state= 19)
r_squares = cross_val_score(lasso_model, feature, price, cv = fold_10, scoring = make_scorer(R2))

# Collect the R_squared scores and find the mean R_squared score
print("R-squares:\n", r_squares, "\n")
print("Average R-squares:\n", np.mean(r_squares),'\n\n\n')


# Now we will evaluate the mean squared error
mse = cross_val_score(lasso_model, feature, price, cv = fold_10, scoring = make_scorer(mean_squared_error))

# Collect the R_squared scores and find the mean R_squared score
print("Mean Squared Error:\n", mse, "\n")
print("Average Mean Squared Error:\n", np.mean(mse))

R-squares:
 [0.7259361  0.49559302 0.22667815 0.53759465 0.66090186 0.47154913
 0.61584223 0.51914523 0.45583425 0.65702113] 

Average R-squares:
 0.5366095753498312 



Mean Squared Error:
 [8.83351833e+10 1.55206827e+11 1.52451571e+11 1.49630221e+11
 9.57392064e+10 1.23147722e+11 1.65120368e+11 4.87656436e+11
 2.20433860e+11 1.54305769e+11] 

Average Mean Squared Error:
 179202716536.7052


/Users/eddyyu/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.586e+12, tolerance: 1.441e+10
  model = cd_fast.enet_coordinate_descent(
/Users/eddyyu/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.481e+12, tolerance: 1.450e+10
  model = cd_fast.enet_coordinate_descent(
/Users/eddyyu/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularis

By comparing average R^2 results and average Mean Squared Error from both linear regression and ridge models, the ridge model with alpha being set to 2 does not necessay make the performance of the mode any better.

### 5. Out-of-sample performance (25 pts):
- Ignore all previously trained models.

- Split the data into a new training and test 80-20.

- Newly train (fit) the linear regression model on the training data.

- Newly train (fit) the lasso regression model on the training data.

- Estimate AIC, AICc, BIC, as well as 5-fold CV for both models using only the training data.

- Estimate the models true OOS performance by computing their deviance on the test data.

- Compare all (deviance) values.  Which IC is most similar to the models’ true OOS performance?  How does 5-fold CV compare?  Explain.

In [19]:
# Split the data into a new training and test 80-20
x_train, x_test, y_train, y_test = train_test_split(feature, price, test_size=0.2, random_state=22)

In [20]:
# Linear Regression Model
model = sm.OLS(y_train,sm.add_constant(x_train)).fit()

# Lasso Regression Model
model_lasso = Lasso(alpha=2.0)
model_lasso.fit(x_train, y_train)

/Users/eddyyu/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.173e+12, tolerance: 1.285e+10
  model = cd_fast.enet_coordinate_descent(


Lasso(alpha=2.0)

In [21]:
# Estimate AIC, AICc, BIC, as well as 5-fold CV for linear regression model using only the training data.

# aic
aic = model.aic

# aicc
n_obs = len(y_train)
n_parameter = model.df_model + 1
aicc = aic + (2 * n_parameter * (n_parameter+1))/(n_obs - n_parameter - 1)

#bic
bic = model.bic

print(f'AIC is {aic}') 
print(f'AICc is {aicc}') 
print(f'BIC is {bic}') 

# 5-fold cv
fold_5 = KFold(n_splits=5, shuffle = True, random_state= 12)
model_MSRE = cross_val_score(LinearRegression(), x_train, y_train, cv = fold_5, scoring = make_scorer(mean_squared_error))
model_MSRE = np.mean(model_MSRE)
print(f'Mean Squared Error for Linear Regression model using training data is {model_MSRE}.')

AIC is 8969.189864871452
AICc is 8971.033932668062
BIC is 9029.077915876403
Mean Squared Error for Linear Regression model using training data is 177216789326.17624.


In [24]:
# Estimate AIC, AICc, BIC, as well as 5-fold CV for Lasso regression model using only the training data. we have to mannaully calculate Aic, Aicc and Bic

# find mse from training data
y_pred_train = model_lasso.predict(x_train)
sse = np.sum((y_pred_train - y_train)**2)

# getting k and n ready for calculating AIC, AICc and BIC
k = len(model_lasso.coef_) + 1
n = len(y_train)

# AIC
aic_lasso = n * np.log(sse / n) + 2 * k

# AICc
aicc_lasso = aic_lasso + (2 * k * (k + 1)) / (n - k - 1)

# BIC
bic_lasso = n * np.log(sse / n) + k * np.log(n)

print(f'AIC for Lasso model is {aic_lasso}') 
print(f'AICc for Lasso model is {aicc_lasso}') 
print(f'BIC for Lasso model is {bic_lasso}') 

# 5-fold cv
fold_5 = KFold(n_splits=5, shuffle = True, random_state= 12)
model_lasso_MSRE = cross_val_score(model_lasso, x_train, y_train, cv = fold_5, scoring = make_scorer(mean_squared_error))
model_lasso_MSRE = np.mean(model_lasso_MSRE)
print(f'Mean Squared Error for Lasso Regression model using training data is {model_lasso_MSRE}.')

AIC for Lasso model is 8089.772225563693
AICc for Lasso model is 8092.37496528972
BIC for Lasso model is 8160.889286132073
Mean Squared Error for Lasso Regression model using training data is 177212498458.1553.


/Users/eddyyu/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.306e+12, tolerance: 1.006e+10
  model = cd_fast.enet_coordinate_descent(
/Users/eddyyu/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.627e+12, tolerance: 9.447e+09
  model = cd_fast.enet_coordinate_descent(
/Users/eddyyu/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularis

In [25]:
# Estimate the models true OOS performance by computing their deviance on the test data.

# linear rergession model
y_pred_test_linear = model.predict(sm.add_constant(x_test))
linear_model_deviance = deviance(y_test, y_pred_test_linear)

# Ridge rergession model
y_pred_test_lasso = model_lasso.predict(x_test)
lasso_model_deviance = deviance(y_test, y_pred_test_lasso)

print(f'The deviance on the test data for Linear regression model is {linear_model_deviance}.','\n')
print(f'The deviance on the test data for Ridge regression model is {lasso_model_deviance}.')

The deviance on the test data for Linear regression model is 13776307894088.816. 

The deviance on the test data for Ridge regression model is 13803357137248.117.


By comparinig the deviances calucated from both linear regression model and ridge model using test data, we can find that deviance from ridge model is much smaller than that of linear regression model. This finding can also be applied to the AIC, AICc and BIC of both models. Ridge model seems to lower the AIC, AICc and BIC values compared to Linear Regression model. However, the mean squared errors calculated through 5-cross-validation for both models seem to stay unchanged. Overall, by using Ridge Model with an alpha value of 2.0, we can increase our model prediction accuracy.